### Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

### Loading Data

In [ ]:
train_data=np.load('ratings_train.npy')
test_data=np.load('ratings_test.npy')
train_data[np.isnan(train_data)] = 0
test_data[np.isnan(test_data)] = 0

In [ ]:
def matrix_to_triplet(data):
    users, items = data.shape
    user_ids, item_ids, ratings = [], [], []

    for i in range(users):
        for j in range(items):
            if (data[i, j]>0):  # if the rating is not zero
                user_ids.append(i)
                item_ids.append(j)
                ratings.append(data[i, j])

    return pd.DataFrame({
        'userID': user_ids,
        'itemID': item_ids,
        'rating': ratings
    }),user_ids,item_ids

train_triplet,user_ids,item_ids = matrix_to_triplet(train_data)

### Defining functions

In [ ]:
n_users=train_data.shape[0]
n_movies=train_data.shape[1]
n_dims = 10
user_to_row = {i: i for i in range(n_users)}
movie_to_column = {j: j for j in range(n_movies)}
R = train_data.copy()



In [ ]:
parameters = {}

#### Initialize Parameters

In [ ]:
def initialize_parameters(lambda_U, lambda_V):
    U = np.zeros((n_dims, n_users), dtype=np.float64)
    V = np.random.normal(0.0, 1.0 / lambda_V, (n_dims, n_movies))

    parameters['U'] = U
    parameters['V'] = V
    parameters['lambda_U'] = lambda_U
    parameters['lambda_V'] = lambda_V

#### Parameter Update

In [ ]:
def update_parameters():
    U = parameters['U']
    V = parameters['V']
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']

    for i in range(n_users):
        V_j = V[:, R[i, :] > 0]
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_U * np.identity(n_dims)), np.dot(R[i, R[i, :] > 0], V_j.T))

    for j in range(n_movies):
        U_i = U[:, R[:, j] > 0]
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_V * np.identity(n_dims)), np.dot(R[R[:, j] > 0, j], U_i.T))

    parameters['U'] = U
    parameters['V'] = V

In [ ]:

def update_parameters():
    U = parameters['U']
    V = parameters['V']
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']

    for i in range(n_users):
        V_j = V[:, R[i, :] > 0]
        #print("V_j shape:", V_j.shape)
        #print("R[i, R[i, :] > 0] shape:", R[i, R[i, :] > 0].shape)
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_U * np.identity(n_dims)), np.dot(R[i, R[i, :] > 0], V_j.T))

    for j in range(n_movies):
        U_i = U[:, R[:, j] > 0]
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_V * np.identity(n_dims)), np.dot(R[R[:, j] > 0, j], U_i.T))

    parameters['U'] = U
    parameters['V'] = V

#### Defining log-posterior

In [ ]:
def log_a_posteriori():
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']
    U = parameters['U']
    V = parameters['V']

    UV = np.dot(U.T, V)
    R_UV = (R[R > 0] - UV[R > 0])

    return -0.5 * (np.sum(np.dot(R_UV, R_UV.T)) + lambda_U * np.sum(np.dot(U, U.T)) + lambda_V * np.sum(np.dot(V, V.T)))


#### Prediction and evaluation function

The predict function allows us to predict the rating value given the user_id and the movie_id parameters. The value has been scaled within the range 0-5

In [ ]:
def predict(user_id, movie_id):
    U = parameters['U']
    V = parameters['V']

    r_ij = U[:, user_to_row[user_id]].T.reshape(1, -1) @ V[:, movie_to_column[movie_id]].reshape(-1, 1)

    max_rating = parameters['max_rating']
    min_rating = parameters['min_rating']

    return 0 if max_rating == min_rating else ((r_ij[0][0] - min_rating) / (max_rating - min_rating)) * 5.0


In [ ]:
def evaluate(train_data, test_data):
    ground_truths = []
    predictions = []

    # Iterate through the test dataset
    for i in range(test_data.shape[0]):
        for j in range(test_data.shape[1]):
            if (test_data[i, j]>0):  # Include if test is not NaN and train is NaN or 0
                ground_truths.append(test_data[i, j])
                predictions.append(predict(i, j))  # Use indices for user and movie

    # Calculate RMSE only if there are predictions
    if ground_truths:
        return mean_squared_error(ground_truths, predictions, squared=False)
    else:
        return float('nan')  # If there are no valid predictions, return NaN


In [ ]:
def update_max_min_ratings():
    U = parameters['U']
    V = parameters['V']

    R = U.T @ V
    min_rating = np.min(R)
    max_rating = np.max(R)

    parameters['min_rating'] = min_rating
    parameters['max_rating'] = max_rating

#### Training function

In [ ]:
def train(n_epochs):
    initialize_parameters(0.3, 0.3)
    log_aps = []
    #rmse_train = []
    rmse_test = []

    update_max_min_ratings()
    #rmse_train.append(evaluate(train_data, train_data))  # Training evaluation
    rmse_test.append(evaluate(train_data, test_data))    # Testing evaluation

    for k in range(n_epochs):
        update_parameters()
        log_ap = log_a_posteriori()
        log_aps.append(log_ap)

        if (k + 1) % 10 == 0:
            update_max_min_ratings()

            #rmse_train.append(evaluate(train_data, train_data))
            rmse_test.append(evaluate(train_data, test_data))
            print('Log p a-posteriori at iteration', k + 1, ':', log_ap)

    update_max_min_ratings()

    return log_aps, rmse_test


In [ ]:
log_ps, rmse_test = train(100)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Log p a-posteriori at iteration 10 : -12008.489918351697


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Log p a-posteriori at iteration 20 : -5229.911688410699


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Log p a-posteriori at iteration 80 : -3494.2299125202317


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Log p a-posteriori at iteration 90 : -3592.90070985049
Log p a-posteriori at iteration 100 : -3676.4552728290664


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
min(rmse_test)

1.195902728852581

### Hyperparameter Tuning

#### Data Split

In [ ]:
def train_val_split_dense(train_data, val_fraction=0.1, random_seed=42):
    """
    Split the given dense train matrix into training and validation matrices by masking some entries.

    Args:
    - train_data: The original dense training matrix.
    - val_fraction: Fraction of entries to mask for validation.
    - random_seed: Random seed for reproducibility.

    Returns:
    - train_data_new: Training matrix with some entries removed.
    - val_data: Validation matrix with the removed entries.
    """
    np.random.seed(random_seed)

    # Create validation matrix initialized with NaNs
    val_data = np.full(train_data.shape, 0)

    # Get the non-NaN entries of the train matrix
    user_indices, item_indices = np.where((train_data>0))
    non_nan_entries = list(zip(user_indices, item_indices))

    # Shuffle and select a subset of these entries for validation
    num_val = int(len(non_nan_entries) * val_fraction)
    val_indices = np.random.choice(len(non_nan_entries), num_val, replace=False)

    # Create a copy of the training data to mask some entries
    train_data_new = train_data.copy()

    # Mask selected entries from training and move them to validation
    for idx in val_indices:
        user, item = non_nan_entries[idx]
        val_data[user, item] = train_data_new[user, item]  # Move to validation set
        train_data_new[user, item] = 0  # Remove from training set

    return train_data_new, val_data



train_data_new, val_data = train_val_split_dense(train_data, val_fraction=0.2, random_seed=42)
print(train_data_new.shape)
print(val_data.shape)

(610, 4980)
(610, 4980)


#### Defining hyperparameterization function

In [ ]:
def train_with_hyperparameters(lambda_U, lambda_V, n_dims, n_epochs=100):
    parameters = {}
    initialize_parameters(lambda_U, lambda_V)
    log_aps = []
    rmse_test = []
    update_max_min_ratings()
    rmse_test.append(evaluate(train_data_new, val_data))

    for k in range(n_epochs):
        update_parameters()
        log_ap = log_a_posteriori()
        log_aps.append(log_ap)

        if (k + 1) % 10 == 0:
            update_max_min_ratings()
            rmse_test.append(evaluate(train_data_new, val_data))


    update_max_min_ratings()

    return log_aps, rmse_test[-1]






# Hyperparameter tuning using grid search
def hyperparameter_tuning(lambdas_U, lambdas_V, n_dims_list, n_epochs=100):
    best_params = {}
    best_rmse = float('inf')
    rmse_val = []

    for lambda_U in lambdas_U:
        for lambda_V in lambdas_V:
            for n_dims in n_dims_list:
                print(f"Testing: lambda_U={lambda_U}, lambda_V={lambda_V}, n_dims={n_dims}")
                log_aps, rmse_test = train_with_hyperparameters(lambda_U, lambda_V, n_dims, n_epochs)
                rmse_val.append(rmse_test)
                print(f"Validation RMSE: {rmse_test}")

                if rmse_val[-1] < best_rmse:
                    best_rmse = rmse_val[-1]
                    best_params = {
                        'lambda_U': lambda_U,
                        'lambda_V': lambda_V,
                        'n_dims': n_dims
                    }

    return best_params, best_rmse



In [ ]:
lambdas_U = [0.1, 0.3, 0.5]
lambdas_V = [0.1, 0.3, 0.5]
n_dims_list = [5, 10, 20]

# Run grid search
best_params, best_rmse = hyperparameter_tuning(lambdas_U, lambdas_V, n_dims_list, n_epochs=100)
print("Best hyperparameters:", best_params)
print("Best RMSE on test set:", best_rmse)

Testing: lambda_U=0.1, lambda_V=0.1, n_dims=5
Validation RMSE: 1.0669675540560475
Testing: lambda_U=0.1, lambda_V=0.1, n_dims=10
Validation RMSE: 1.1018290198810134
Testing: lambda_U=0.1, lambda_V=0.1, n_dims=20
Validation RMSE: 0.9508107599669322
Testing: lambda_U=0.1, lambda_V=0.3, n_dims=5
Validation RMSE: 1.2089422206670462
Testing: lambda_U=0.1, lambda_V=0.3, n_dims=10
Validation RMSE: 1.1532339482622627
Testing: lambda_U=0.1, lambda_V=0.3, n_dims=20
Validation RMSE: 1.1140352152087492
Testing: lambda_U=0.1, lambda_V=0.5, n_dims=5
Validation RMSE: 1.135891537628939
Testing: lambda_U=0.1, lambda_V=0.5, n_dims=10
Validation RMSE: 1.0653793559667106
Testing: lambda_U=0.1, lambda_V=0.5, n_dims=20
Validation RMSE: 1.0235756444318376
Testing: lambda_U=0.3, lambda_V=0.1, n_dims=5
Validation RMSE: 1.0181649114607256
Testing: lambda_U=0.3, lambda_V=0.1, n_dims=10
Validation RMSE: 1.199608277497845
Testing: lambda_U=0.3, lambda_V=0.1, n_dims=20
Validation RMSE: 0.9057302879234053
Testing: l

### Final training and evaluation

In [ ]:
def final_train(lambda_U, lambda_V, n_dims, n_epochs=100):
    parameters = {}
    initialize_parameters(lambda_U, lambda_V)
    log_aps = []
    rmse_test = []
    update_max_min_ratings()
    rmse_test.append(evaluate(train_data, test_data))

    for k in range(n_epochs):
        update_parameters()
        log_ap = log_a_posteriori()
        log_aps.append(log_ap)

        if (k + 1) % 10 == 0:
            update_max_min_ratings()
            rmse_test.append(evaluate(train_data_new, val_data))


    update_max_min_ratings()

    return log_aps, rmse_test[-1]

log_aps, rmse_test_final = final_train(0.5,0.1, 5, 100)

In [ ]:
print(rmse_test_final)

1.0349574091720315
